In [ ]:
import yt
from yt.utilities.amr_kdtree.api import AMRKDTree
from yt.data_objects import *
import numpy as np
import sympy as sp

In [ ]:
def floatingIndex(x, L, N):
    dx = L/N
    
    return (x+L/2)/dx - 1/2

def ceilIndex(x, L, N):
    dx = L/N
    return int(np.ceil(floatingIndex(x,L,N)))

def floorIndex(x, L, N):
    dx = L/N
    return int(np.floor(floatingIndex(x,L,N)))

def pointvalIndex(n, L, N):
    dx = L/N
    return -L/2 + (n+1/2)*dx

def trilinearInterpolation(xpt, ypt, zpt, L, N, ds, var):

    nx = floatingIndex(xpt,L,N)
    nx0 = floorIndex(xpt,L,N)
    nx1 = ceilIndex(xpt,L,N)

    ny = floatingIndex(ypt,L,N)
    ny0 = floorIndex(ypt,L,N)
    ny1 = ceilIndex(ypt,L,N)

    nz = floatingIndex(zpt,L,N)
    nz0 = floorIndex(zpt,L,N)
    nz1 = ceilIndex(zpt,L,N)
    
    xd = (nx-nx0)/(nx1-nx0)
    yd = (ny-ny0)/(ny1-ny0) 
    zd = (nz-nz0)/(nz1-nz0)
    
    c000 = float(str(ds[('boxlib',var)][nx0,ny0,nz0])[0:-14])
    c100 = float(str(ds[('boxlib',var)][nx1,ny0,nz0])[0:-14])
    c101 = float(str(ds[('boxlib',var)][nx1,ny0,nz1])[0:-14])
    c001 = float(str(ds[('boxlib',var)][nx0,ny0,nz1])[0:-14])

    c010 = float(str(ds[('boxlib',var)][nx0,ny1,nz0])[0:-14])
    c110 = float(str(ds[('boxlib',var)][nx1,ny1,nz0])[0:-14])
    c111 = float(str(ds[('boxlib',var)][nx1,ny1,nz1])[0:-14])
    c011 = float(str(ds[('boxlib',var)][nx0,ny1,nz1])[0:-14])
    
    c00 = c000*(1-xd) + c100*xd
    c01 = c001*(1-xd) + c101*xd
    c10 = c010*(1-xd) + c110*xd
    c11 = c011*(1-xd) + c111*xd

    c0 = c00*(1-yd) + c10*yd
    c1 = c01*(1-yd) + c11*yd
    
    return c0*(1-zd) + c1*zd

In [ ]:
valrelist = []
valre4list = []
valimlist = []
timelist = []

r = 1000.0
NumTheta = 20
NumPhi = 10
hTheta = np.pi/NumTheta
hPhi = 2*np.pi/NumPhi



for l in range(400,400):
    t = 1.6*l
    current_time = (t-r) #(t-r)/M
    plot_num = l
    diag_num = plot_num
    plot_num += 1000000
    diag_num += 1000000
    plot_num_list = list(str(plot_num))
    diag_num_list = list(str(diag_num))
    plot_num_list[0] = '0'
    diag_num_list[0] = '0'
    #plotname = "plt"+"".join(plot_num_list)
    diagname = "diag"+"".join(diag_num_list)
    
    #dplot = yt.load(plotname)
    ddiag = yt.load(diagname)
    d = ddiag.r
    L = 4096
    N = 256*2
    testval = 0
    testval2 = 0
    integralre = 0
    integralre2 = 0
    integralim = 0
    integralim2 = 0
    for i in range(NumTheta+1):
        for j in range(NumPhi+1):
            theta = i*hTheta
            phi = j*hPhi
            xpt = r*np.sin(theta)*np.cos(phi)
            ypt = r*np.sin(theta)*np.sin(phi)
            zpt = r*np.cos(theta)
            pt = d[xpt,ypt,zpt]
            #valptre = trilinearInterpolation(xpt, ypt, zpt, L, N, ddiag, 'Psi4re')
            #valptim = trilinearInterpolation(xpt, ypt, zpt, L, N, ddiag, 'Psi4im')
            valptre = float(str(pt[('boxlib','Psi4re')][0])[0:-14])
            valptim = float(str(pt[('boxlib','Psi4im')][0])[0:-14])
            
            factorTheta = (i == 0)*1/3 + (i == NumTheta)*1/3 + (i%2 == 0)*(i != 0)*(i != NumTheta)*2/3 + (i%2 == 1)*4/3
            factorPhi = (j == 0)*1/3 + (j == NumPhi)*1/3 + (j%2 == 0)*(j != 0)*(j != NumPhi)*2/3 + (j%2 == 1)*4/3
        
            #integralre += (r/M)*np.sqrt(5/(4*np.pi))*(np.cos(theta/2)**4)*(np.cos(2*phi)*valptre + np.sin(2*phi)*valptim)*np.sin(theta)*hTheta*hPhi*factorTheta*factorPhi
            #integralim += (r/M)*np.sqrt(5/(4*np.pi))*(np.cos(theta/2)**4)*(np.sin(2*phi)*valptre - np.cos(2*phi)*valptim)*np.sin(theta)*hTheta*hPhi*factorTheta*factorPhi
            testval += hTheta*hPhi*factorTheta*factorPhi*((3.0/4.0)*np.sqrt(5.0/(6.0*np.pi)))**2*(np.sin(theta)**4)*np.sin(theta)
            testval2 += hTheta*hPhi*factorTheta*factorPhi*np.sin(theta)*((3.0/16.0)*np.sqrt(5.0/(2*np.pi))*(5+7*np.cos(2*theta))*(np.sin(theta)**2))**2
            integralre += (r)*(3.0/4.0)*np.sqrt(5.0/(6.0*np.pi))*(np.sin(theta)**2)*(valptre)*np.sin(theta)*hTheta*hPhi*factorTheta*factorPhi
            integralre2 += r*(3.0/16.0)*np.sqrt(5.0/(2*np.pi))*(5+7*np.cos(2*theta))*(np.sin(theta)**2)*(valptre)*np.sin(theta)*hTheta*hPhi*factorTheta*factorPhi
            #integralim += (r)*(3.0/4.0)*np.sqrt(5.0/(6.0*np.pi))*(np.sin(theta)**2)*(valptim)*np.sin(theta)*hTheta*hPhi
            
    valrelist.append(integralre)
    valre4list.append(integralre2)
    valimlist.append(integralim)
    timelist.append(current_time)
    print(testval)
    print(testval2)
    print(integralre)
    print(integralre2)
    print(integralim)
    print(" "+str(current_time)+" "+str(l))

In [ ]:
valrelist = []
valre4list = []
valimlist = []
timelist = []

r = 700.1  #Apparently can't be rational?
NumTheta = 20
NumPhi = 10
hTheta = np.pi/NumTheta
hPhi = 2*np.pi/NumPhi



for l in range(0,1001):
    t = 1.6*l
    current_time = (t-r) #(t-r)/M
    plot_num = l
    diag_num = plot_num
    plot_num += 1000000
    diag_num += 1000000
    plot_num_list = list(str(plot_num))
    diag_num_list = list(str(diag_num))
    plot_num_list[0] = '0'
    diag_num_list[0] = '0'
    #plotname = "plt"+"".join(plot_num_list)
    diagname = "diag"+"".join(diag_num_list)
    
    #dplot = yt.load(plotname)
    ddiagLO = yt.load(diagname)
    ddiag = ddiagLO.covering_grid(1,[-2048.0,-2048.0,-2048.0], ddiagLO.domain_dimensions*2)
    d = ddiagLO.r
    L = 4096
    N = 128*2
    testval = 0
    testval2 = 0
    integralre = 0
    integralre2 = 0
    integralim = 0
    integralim2 = 0
    for i in range(NumTheta+1):
        for j in range(NumPhi+1):
            theta = i*hTheta
            phi = j*hPhi
            xpt = r*np.sin(theta)*np.cos(phi)
            ypt = r*np.sin(theta)*np.sin(phi)
            zpt = r*np.cos(theta)
            pt = d[xpt,ypt,zpt]
            valptre = trilinearInterpolation(xpt, ypt, zpt, L, N, ddiag, 'Psi4re')
            valptim = trilinearInterpolation(xpt, ypt, zpt, L, N, ddiag, 'Psi4im')
            #valptre = float(str(pt[('boxlib','Psi4re')][0])[0:-14])
            #valptim = float(str(pt[('boxlib','Psi4im')][0])[0:-14])
            
            factorTheta = (i == 0)*1/3 + (i == NumTheta)*1/3 + (i%2 == 0)*(i != 0)*(i != NumTheta)*2/3 + (i%2 == 1)*4/3
            factorPhi = (j == 0)*1/3 + (j == NumPhi)*1/3 + (j%2 == 0)*(j != 0)*(j != NumPhi)*2/3 + (j%2 == 1)*4/3
        
            #integralre += (r/M)*np.sqrt(5/(4*np.pi))*(np.cos(theta/2)**4)*(np.cos(2*phi)*valptre + np.sin(2*phi)*valptim)*np.sin(theta)*hTheta*hPhi*factorTheta*factorPhi
            #integralim += (r/M)*np.sqrt(5/(4*np.pi))*(np.cos(theta/2)**4)*(np.sin(2*phi)*valptre - np.cos(2*phi)*valptim)*np.sin(theta)*hTheta*hPhi*factorTheta*factorPhi
            testval += hTheta*hPhi*factorTheta*factorPhi*((3.0/4.0)*np.sqrt(5.0/(6.0*np.pi)))**2*(np.sin(theta)**4)*np.sin(theta)
            testval2 += hTheta*hPhi*factorTheta*factorPhi*np.sin(theta)*((3.0/16.0)*np.sqrt(5.0/(2*np.pi))*(5+7*np.cos(2*theta))*(np.sin(theta)**2))**2
            integralre += (r)*(3.0/4.0)*np.sqrt(5.0/(6.0*np.pi))*(np.sin(theta)**2)*(valptre)*np.sin(theta)*hTheta*hPhi*factorTheta*factorPhi
            integralre2 += r*(3.0/16.0)*np.sqrt(5.0/(2*np.pi))*(5+7*np.cos(2*theta))*(np.sin(theta)**2)*(valptre)*np.sin(theta)*hTheta*hPhi*factorTheta*factorPhi
            #integralim += (r)*(3.0/4.0)*np.sqrt(5.0/(6.0*np.pi))*(np.sin(theta)**2)*(valptim)*np.sin(theta)*hTheta*hPhi
            
    valrelist.append(integralre)
    valre4list.append(integralre2)
    valimlist.append(integralim)
    timelist.append(current_time)
    print(testval)
    print(testval2)
    print(integralre)
    print(integralre2)
    print(integralim)
    print(" "+str(current_time)+" "+str(l))

In [ ]:
for i in range(len(valrelist)):
    print('{' + str(timelist[i]*10000) + ', ' + str(valre4list[i]*10000) + '},')

In [ ]:
for i in range(len(valrelist)):
    print('{' + str(timelist[i]*10000) + ', ' + str(valre4list[i]*10000) + '},')

In [ ]:
valrelist = []
valimlist = []
timelist = []


r = 200.0
NumTheta = 20
NumPhi = 1
hTheta = np.pi/NumTheta
hPhi = 0.25*np.pi/NumPhi

for l in range(1,301):
    t = 3.2*l
    current_time = t #(t-r)/M
    plot_num = l
    diag_num = plot_num
    plot_num += 1000000
    diag_num += 1000000
    plot_num_list = list(str(plot_num))
    diag_num_list = list(str(diag_num))
    plot_num_list[0] = '0'
    diag_num_list[0] = '0'
    #plotname = "plt"+"".join(plot_num_list)
    diagname = "diag"+"".join(diag_num_list)
    
    #dplot = yt.load(plotname)
    ddiagLO = yt.load(diagname)
    ddiag = ddiagLO.smoothed_covering_grid(3,[-1024.0,-1024.0,-1024.0], ddiagLO.domain_dimensions*2**3)
    d = ddiagLO.r
    L = 2048
    N = 64*2**3
    #d = ddiag.r
    integralre = 0
    #integralre2 = 0
    #integralim = 0
    #integralim2 = 0
    for i in range(NumTheta+1):
        for j in range(NumPhi + 1):
            theta = i*hTheta
            phi = j*hPhi
            xpt = r*np.sin(theta)*np.cos(phi)
            ypt = r*np.sin(theta)*np.sin(phi)
            zpt = r*np.cos(theta)
            pt = d[xpt,ypt,zpt]
            valptre = trilinearInterpolation(xpt, ypt, zpt, L, N, ddiag, 'Psi4re')
            #valptre = float(str(pt[('boxlib','Psi4re')][0])[0:-14])
            valptim = trilinearInterpolation(xpt, ypt, zpt, L, N, ddiag, 'Psi4im')
            
            factorTheta = (i == 0)*1/3 + (i == NumTheta)*1/3 + (i%2 == 0)*(i != 0)*(i != NumTheta)*2/3 + (i%2 == 1)*4/3
            factorPhi = (j == 0)*1/3 + (j == NumPhi)*1/3 + (j%2 == 0)*(j != 0)*(j != NumPhi)*2/3 + (j%2 == 1)*4/3
        
            #integralre += (r/M)*np.sqrt(5/(4*np.pi))*(np.cos(theta/2)**4)*(np.cos(2*phi)*valptre + np.sin(2*phi)*valptim)*np.sin(theta)*hTheta*hPhi*factorTheta*factorPhi
            #integralim += (r/M)*np.sqrt(5/(4*np.pi))*(np.cos(theta/2)**4)*(np.sin(2*phi)*valptre - np.cos(2*phi)*valptim)*np.sin(theta)*hTheta*hPhi*factorTheta*factorPhi
            
            integralre += 2*(r)*(3.0/4.0)*np.sqrt(5.0/(6.0*np.pi))*(np.sin(theta)**2)*(valptre)*np.sin(theta)*hTheta*factorTheta*hPhi*factorPhi 
            #integralim += 8*(r/M)*(3.0/4.0)*np.sqrt(5.0/(6.0*np.pi))*(np.sin(theta)**2)*(valptim)*np.sin(theta)*hTheta*factorTheta*hPhi*factorPhi 
    
        
    valrelist.append(integralre)
    #valimlist.append(integralim)
    #timelist.append(current_time)
    print(integralre)
    #print(integralim)
    print(" "+str(current_time)+" "+str(l))

In [ ]:
valrelist = []
valimlist = []
timelist = []

r = 1500.0
NumTheta = 20
NumPhi = 10
hTheta = np.pi/NumTheta
hPhi = 2*np.pi/NumPhi

for l in range(468,468+260):
    t = 3.2*l
    current_time = (t-r) #(t-r)/M
    plot_num = l
    diag_num = plot_num
    plot_num += 1000000
    diag_num += 1000000
    plot_num_list = list(str(plot_num))
    diag_num_list = list(str(diag_num))
    plot_num_list[0] = '0'
    diag_num_list[0] = '0'
    #plotname = "plt"+"".join(plot_num_list)
    diagname = "diag"+"".join(diag_num_list)
    
    #dplot = yt.load(plotname)
    ddiag = yt.load(diagname)
    d = ddiag.r
    integralre = 0
    integralre2 = 0
    integralim = 0
    integralim2 = 0
    theta = np.pi/2.0 #i*hTheta
    phi = 0 #j*hPhi
    xpt = r*np.sin(theta)*np.cos(phi)
    ypt = r*np.sin(theta)*np.sin(phi)
    zpt = r*np.cos(theta)
    pt = d[xpt,ypt,zpt]
    valptre = r*float(str(pt[('boxlib','Psi4re')][0])[0:-14])
    valptim = r*float(str(pt[('boxlib','Psi4im')][0])[0:-14])
            
    #factorTheta = (i == 0)*1/3 + (i == NumTheta)*1/3 + (i%2 == 0)*(i != 0)*(i != NumTheta)*2/3 + (i%2 == 1)*4/3
    #factorPhi = (j == 0)*1/3 + (j == NumPhi)*1/3 + (j%2 == 0)*(j != 0)*(j != NumPhi)*2/3 + (j%2 == 1)*4/3
        
            #integralre += (r/M)*np.sqrt(5/(4*np.pi))*(np.cos(theta/2)**4)*(np.cos(2*phi)*valptre + np.sin(2*phi)*valptim)*np.sin(theta)*hTheta*hPhi*factorTheta*factorPhi
            #integralim += (r/M)*np.sqrt(5/(4*np.pi))*(np.cos(theta/2)**4)*(np.sin(2*phi)*valptre - np.cos(2*phi)*valptim)*np.sin(theta)*hTheta*hPhi*factorTheta*factorPhi
            
    #integralre += 2*np.pi*(r/9)*(3.0/4.0)*np.sqrt(5.0/(6.0*np.pi))*(np.sin(theta)**2)*(valptre)*np.sin(theta)*hTheta*hPhi*factorTheta*factorPhi
            #integralim += 2*np.pi*(r/9)*(3.0/4.0)*np.sqrt(5.0/(6.0*np.pi))*(np.sin(theta)**2)*(valptim)*np.sin(theta)*hTheta*hPhi
            
    valrelist.append(valptre)
    #valimlist.append(integralim)
    timelist.append(current_time)
    #print(integralre)
    #print(integralim)
    print(" "+str(current_time)+" "+str(l))

In [ ]:
for i in range(len(valrelist)):
    print('{' + str(timelist[i]*1000) + ', ' + str(valrelist[i]*1000) + '},')